# 📘 word_full_pipeline_v8.1 — 完整自动构建版

改进点：
- 自动降级 `docutils<0.21` 兼容 Sphinx 8.x；
- 自动转义 RST 文件名（AT+ → at_plus 等）；
- 自动生成完整 conf.py（含主题、CSS、扩展）；
- 自动检测并输出 HTML 构建结果。

> 请确保 `AT_Commands.docx` 在同一目录下。

In [20]:
!pip install -q python-docx pandas pyyaml jinja2 sphinx sphinx-book-theme sphinx-copybutton "docutils<0.21" lxml
print('✅ 所有依赖安装完成。')

✅ 所有依赖安装完成。


In [21]:
import os, re, json, shutil, subprocess, sys, datetime, yaml
import pandas as pd
from docx import Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from jinja2 import Template

# === 路径配置 ===
IN_WORD = 'AT_Commands.docx'
DATA_DIR = 'data'
CSV_OUT = os.path.join(DATA_DIR, 'commands.csv')
YAML_OUT = os.path.join(DATA_DIR, 'commands.yaml')
RST_DIR = os.path.join(DATA_DIR, 'rst_output')

DOCS_ROOT = 'docs'
LANG = 'zh_CN'
SRC_DIR = os.path.join(DOCS_ROOT, LANG, 'source')
BUILD_HTML = os.path.join(DOCS_ROOT, LANG, 'build', 'html')
LOG_PATH = 'parse_log.txt'

os.makedirs(DATA_DIR, exist_ok=True)
open(LOG_PATH,'w').close()

def log(msg):
    with open(LOG_PATH,'a',encoding='utf-8') as f:
        f.write(f"[{datetime.datetime.now().isoformat()}] {msg}\n")

print('✅ 配置完成。')

✅ 配置完成。


## Step 1 — Word → CSV（示例）

In [22]:
def extract_word_to_csv(docx_path, csv_out):
    doc = Document(docx_path)
    rows = []
    for p in doc.paragraphs:
        if p.text.strip().startswith('AT+'):
            cmd = p.text.strip()
            rows.append({
                '命令': cmd,
                '命令标题': 'Auto Extracted',
                '命令类型': '执行',
                '命令格式': cmd,
                '示例命令': cmd,
                '示例响应': 'OK',
                '功能描述': '自动提取的示例命令',
                '备注': '',
                '参数JSON': '[]'
            })
    df = pd.DataFrame(rows)
    df.to_csv(csv_out,index=False,encoding='utf-8-sig')
    print(f'✅ 已提取 {len(df)} 条命令 → {csv_out}')
    return df

_ = extract_word_to_csv(IN_WORD, CSV_OUT)

✅ 已提取 41 条命令 → data\commands.csv


## Step 2 — CSV → YAML

In [23]:
def csv_to_yaml(csv_path, yaml_path):
    df = pd.read_csv(csv_path, dtype=str).fillna('')
    data = []
    for _, r in df.iterrows():
        params = json.loads(r.get('参数JSON','[]'))
        data.append({
            'command': r['命令'],
            'title': r['命令标题'],
            'type': [r['命令类型']],
            'formats': [r['命令格式']],
            'parameters': params,
            'examples': [{'cmd': r['示例命令'], 'resp': r['示例响应']}],
            'description': r['功能描述'],
            'notes': r['备注']
        })
    with open(yaml_path, 'w', encoding='utf-8') as f:
        yaml.safe_dump({'commands': data}, f, allow_unicode=True)
    print(f'✅ 已生成 YAML → {yaml_path}')

csv_to_yaml(CSV_OUT, YAML_OUT)

✅ 已生成 YAML → data\commands.yaml


## Step 3 — YAML → 安全命名 RST（带锚点）

In [24]:
RST_TMPL = Template('''
.. _cmd_{{ cmd.command.replace('+','plus') }}:

{{ cmd.command }}
{{ '=' * cmd.command|length }}

**功能描述**
   {{ cmd.description }}

**示例**
.. code-block:: bash

   {{ cmd.examples[0]['cmd'] }}
   {{ cmd.examples[0]['resp'] }}
''')

def yaml_to_rst(yaml_path, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    with open(yaml_path,'r',encoding='utf-8') as f:
        data = yaml.safe_load(f)
    for c in data['commands']:
        safe_name = c['command'].replace('+','plus').replace('?','q').replace('=','eq')
        rst = RST_TMPL.render(cmd=c)
        with open(os.path.join(out_dir,f'{safe_name}.rst'),'w',encoding='utf-8') as f:
            f.write(rst)
    index = ['AT Commands\n===========\n','\n.. toctree::\n   :maxdepth: 1\n']
    for c in data['commands']:
        safe_name = c['command'].replace('+','plus').replace('?','q').replace('=','eq')
        index.append(f'   {safe_name}')
    with open(os.path.join(out_dir,'index.rst'),'w',encoding='utf-8') as f:
        f.write('\n'.join(index))
    print(f'✅ 已生成 RST → {out_dir}')

yaml_to_rst(YAML_OUT, RST_DIR)

✅ 已生成 RST → data\rst_output


## Step 4 — 初始化 Sphinx（带主题与 conf.py）

In [25]:
if os.path.exists(DOCS_ROOT): shutil.rmtree(DOCS_ROOT)
os.makedirs(SRC_DIR, exist_ok=True)
shutil.copytree(RST_DIR, SRC_DIR, dirs_exist_ok=True)

conf_text = '''
project = 'AT Command Manual'
master_doc = 'index'
language = 'zh_CN'
extensions = ['sphinx.ext.autosectionlabel','sphinx_copybutton']
html_theme = 'sphinx_book_theme'
html_static_path = ['_static']
def setup(app):
    app.add_css_file('custom.css')
'''
os.makedirs(os.path.join(SRC_DIR,'_static'),exist_ok=True)
with open(os.path.join(SRC_DIR,'conf.py'),'w',encoding='utf-8') as f:
    f.write(conf_text)
with open(os.path.join(SRC_DIR,'_static','custom.css'),'w',encoding='utf-8') as f:
    f.write('pre {background:#f8f9fa;border-radius:6px;padding:8px;}')
print('✅ 已创建 conf.py 和 CSS。')

✅ 已创建 conf.py 和 CSS。


## Step 5 — 构建 HTML（验证 index.html）

In [28]:
def build_html():
    print('📦 开始构建 HTML...')
    os.makedirs(BUILD_HTML, exist_ok=True)
    cmd = [sys.executable,'-m','sphinx','-b','html',SRC_DIR,BUILD_HTML]
    p = subprocess.run(cmd, capture_output=True, text=True, encoding="utf-8", errors="ignore")
    print(p.stdout or p.stderr)
    index_path = os.path.join(BUILD_HTML,'index.html')
    if os.path.exists(index_path):
        print(f'✅ 构建成功：{index_path}')
    else:
        print('❌ 构建失败，请检查日志。')

build_html()

📦 开始构建 HTML...
Running Sphinx v8.2.3
loading translations [zh_CN]... done
loading pickled environment... The configuration has changed (5 options: 'html_permalinks_icon', 'html_sourcelink_suffix', 'html_static_path', 'html_theme_options', 'templates_path')
done
building [mo]: targets for 0 po files that are out of date
writing output... 
building [html]: targets for 0 source files that are out of date
updating environment: 0 added, 0 changed, 0 removed
reading sources... 
looking for now-outdated files... none found
no targets are out of date.
preparing documents... done
copying assets... 
copying static files... 
Writing evaluated template result to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\pipeline-1009\docs\zh_CN\build\html\_static\basic.css
Writing evaluated template result to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\pipeline-1009\docs\zh_CN\build\html\_static\documentation_options.js
Writing evaluated template result to 

## Step 6 — 一键运行

In [27]:
def run_all(clean=True):
    if clean and os.path.exists(DOCS_ROOT): shutil.rmtree(DOCS_ROOT)
    extract_word_to_csv(IN_WORD, CSV_OUT)
    csv_to_yaml(CSV_OUT, YAML_OUT)
    yaml_to_rst(YAML_OUT, RST_DIR)
    os.makedirs(SRC_DIR, exist_ok=True)
    shutil.copytree(RST_DIR, SRC_DIR, dirs_exist_ok=True)
    build_html()
    print('🎯 全流程完成 → docs/zh_CN/build/html/index.html')

print('✅ 可执行 run_all(clean=True) 一键生成 HTML。')

✅ 可执行 run_all(clean=True) 一键生成 HTML。
